In [1]:
from dotenv import load_dotenv
import os

load_dotenv()  # 同じフォルダの .env を読む

print("OPENAI:", os.getenv("OPENAI_API_KEY") is not None)
print("TAVILY:", os.getenv("TAVILY_API_KEY") is not None)


OPENAI: True
TAVILY: True


In [2]:
# 必要なモジュールをインポート
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from tavily import TavilyClient

# .env を同じフォルダから読み込む
load_dotenv()

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# tavily検索用APIキーの取得
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

# Tavily クライアント生成
tavily = TavilyClient(api_key=TAVILY_API_KEY)

# モデル名
MODEL_NAME = "gpt-4o-mini"


In [3]:
from tavily import TavilyClient
print("OK")


OK


In [4]:
# Tavily 検索結果を返す関数（最新版対応）
def get_search_result(question):
    # すでに作った tavily クライアントを使う
    response = tavily.search(query=question)

    # 結果を辞書のまま返す（JSON文字列にしない）
    return {
        "result": response["results"]
    }


In [5]:
# テスト用コード
ret = get_search_result("東京駅のイベントを教えて")
print(ret)


{'result': [{'url': 'https://www.walkerplus.com/event_list/ar0313/sc309880d/', 'title': '東京駅(東京都)周辺のイベント - ウォーカープラス', 'content': '開催中 2025年8月8日(金)～2026年3月31日(火). 日比谷駅(東京都), 有楽町駅(東京都), 東京駅(東京都), 京橋駅(東京都). * MARUNOUCHI BRIGHT HOLIDAY 2025 (マルノウチ・ブライト・ホリデー2025) 「Disney JOYFUL MOMENTS」. 開催中 2025年11月13日(木)～2026年1月4日(日). 東京駅(東京都), 二重橋前＜丸の内＞駅(東京都), 有楽町駅(東京都), 大手町駅(東京都), 京橋駅(東京都). * 歴史リアル謎解きゲーム「謎の城」in 日本橋「発明家／人斬り-平賀源内-」. 開催中 2025年9月1日(月)～2026年3月1日(日). 日本橋駅(東京都), 京橋駅(東京都), 東京駅(東京都), 宝町駅(東京都), 三越前駅(東京都). 開催中 2025年11月1日(土)～2026年1月15日(木). 東京駅(東京都), 二重橋前駅(東京都), 京橋駅(東京都), 大手町駅(東京都), 有楽町駅(東京都). * TOKYO ILLUMILIA 2025-2026 (東京イルミリア2025-2026). 開催中 2025年11月6日(木)～2026年2月14日(土). * MIDTOWN YAESU CHRISTMAS 2025 (ミッドタウン八重洲クリスマス2025). 開催中 2025年11月13日(木)～2026年2月15日(日). 京橋駅(東京都), 東京駅(東京都), 宝町駅(東京都), 日本橋駅(東京都), 銀座一丁目駅(東京都). 開催中 2025年11月13日(木)～2026年2月15日(日). 二重橋前〈丸の内〉駅(東京都), 東京駅(東京都), 有楽町駅(東京都), 日比谷駅(東京都), 銀座一丁目駅(東京都). 開催中 2025年10月31日(金)～2026年1月8日(木). 開催中 2025年11月20日(木)～12月25日(木). 東京駅(東京都), 二重橋前駅(東京都), 有楽町駅(東京都),

In [2]:
# ツール定義（SDK v2 対応版）
def define_tools():
    print("------define_tools(ツール定義)------")
    return [
        {
            "type": "function",
            "function": {
                "name": "get_search_result",
                "description": "最近1ヶ月のイベント開催予定などネット検索が必要な場合に、質問文の検索結果を取得する",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                            "description": "検索したい質問文（例：'東京駅のイベントを教えて'）"
                        }
                    },
                    "required": ["question"]
                }
            }
        }
    ]


In [3]:
tools = define_tools()
print(tools)



------define_tools(ツール定義)------
[{'type': 'function', 'function': {'name': 'get_search_result', 'description': '最近1ヶ月のイベント開催予定などネット検索が必要な場合に、質問文の検索結果を取得する', 'parameters': {'type': 'object', 'properties': {'question': {'type': 'string', 'description': "検索したい質問文（例：'東京駅のイベントを教えて'）"}}, 'required': ['question']}}}]


In [4]:
# 言語モデルへの質問を行う関数
def ask_question(question, tools):
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "user", "content": question}
        ],
        tools=tools,
        tool_choice="auto",  # モデルが必要ならツールを呼ぶ
    )
    return response


In [9]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from tavily import TavilyClient

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

client = OpenAI(api_key=OPENAI_API_KEY)
tavily = TavilyClient(api_key=TAVILY_API_KEY)

MODEL_NAME = "gpt-4o-mini"


In [11]:
tools = define_tools()
response = ask_question("東京駅のイベントを教えて", tools)
print(response)


------define_tools(ツール定義)------
ChatCompletion(id='chatcmpl-CkQLZvcyF41lTRk7DU9dhLKMl6oK2', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_7iGl9Hnor7UW7kFb4sabgacq', function=Function(arguments='{"question":"東京駅のイベントを教えて"}', name='get_search_result'), type='function')]))], created=1765180401, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_aa07c96156', usage=CompletionUsage(completion_tokens=21, prompt_tokens=92, total_tokens=113, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [12]:
print("load_dotenv:", 'load_dotenv' in globals())
print("client:", 'client' in globals())
print("tavily:", 'tavily' in globals())
print("define_tools:", 'define_tools' in globals())
print("ask_question:", 'ask_question' in globals())


load_dotenv: True
client: True
tavily: True
define_tools: True
ask_question: True


In [13]:
# ツール呼び出しが必要な場合の処理を行う関数（SDK v2 対応）
def handle_tool_call(response, question):
    # ツールコールの取得
    tool = response.choices[0].message.tool_calls[0]
    function_name = tool.function.name

    # 引数（JSON文字列） → Python dict に変換
    arguments = json.loads(tool.function.arguments)

    # 関数の実行（get_search_result など）
    function_response = globals()[function_name](**arguments)

    # ツールの実行結果を JSON 文字列に変換（OpenAI に渡す形式）
    tool_content = json.dumps(function_response, ensure_ascii=False)

    # messages にツール実行結果を追加してモデルに再問い合わせ
    response_after_tool_call = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "user", "content": question},
            response.choices[0].message,  # tool_call の入った元のメッセージ
            {
                "tool_call_id": tool.id,
                "role": "tool",
                "content": tool_content,  # ★ dict ではなく JSON文字列
            },
        ],
    )

    return response_after_tool_call


In [14]:
# ユーザーからの質問を処理する関数
def process_response(question, tools):
    response = ask_question(question, tools)

    if response.choices[0].finish_reason == 'tool_calls':
        # ツール呼出の場合
        final_response = handle_tool_call(response, question)
        return final_response.choices[0].message.content.strip()
    else:
        # 言語モデルが直接回答する場合
        return response.choices[0].message.content.strip()

In [15]:
tools = define_tools()

# 言語モデルが直接回答できる質問
question = "東京都と沖縄県はどちらが広いですか？"
response_message = process_response(question, tools)
print(response_message)

------define_tools(ツール定義)------
東京都と沖縄県の面積を比較すると、沖縄県の方が広いです。

- 東京都の面積は約2,194平方キロメートルです。
- 沖縄県の面積は約2,271平方キロメートルです。

したがって、沖縄県は東京都よりも広い地域を持っています。


In [18]:
# Tavily 検索結果を返す関数（最新版）
def get_search_result(question):
    # Tavily でWeb検索
    response = tavily.search(query=question)

    # 検索結果を dict で返す（JSON文字列ではない）
    return {
        "result": response["results"]
    }


In [19]:
"get_search_result" in globals()


True

In [20]:
tools = define_tools()

question = "東京駅のイベントについて、最近1ヶ月以内の検索結果を教えてください"
response_message = process_response(question, tools)
print(response_message)


------define_tools(ツール定義)------
最近1ヶ月以内の東京駅周辺で開催されているイベント情報をご紹介します。

1. **東京駅周辺のイベント - ウォーカープラス**
   - 開催中のイベントとして、丸の内・東京駅でのイルミネーションや懐かしさと新しさを感じる街のイベントがたくさん掲載されています。
   - 詳しくは [こちら](https://www.walkerplus.com/event_list/ar0313/sc309880d/)をご覧ください。

2. **丸の内・東京駅のイルミネーション特集**
   - 丸の内にある「Celebration Tree」での点灯式や、クリスマスに向けたアフタヌーンティーが楽しめます。
   - 詳細は [るるぶ&more.](https://rurubu.jp/andmore/area/130101)をご覧ください。

3. **今日行ける！東京駅周辺のおすすめイベント**
   - 11月から12月にかけて様々なイベントが開催されます。特にクリスマスシーズンに向けたイルミネーションやイベントが豊富です。
   - 詳細は [Enjoy Tokyo](https://www.enjoytokyo.jp/event/list/sta200101/its04/)をご確認ください。

4. **駅探による東京駅周辺のイベント一覧**
   - 東京駅の近くで行われているイベント情報がまとめられています。具体的な日程や場所について情報を確認できます。
   - 詳しくは [駅探](https://ekitan.com/event/station-2590)をご覧ください。

これらのリンクを参考に、興味のあるイベントに参加してみてください！


In [21]:
# チャットボットへの組み込み
tools = define_tools()

messages=[]

while(True):
    # ユーザーからの質問を受付
    question = input("メッセージを入力:")
    # 質問が入力されなければ終了
    if question.strip()=="":
        break
    display(f"質問:{question}")

    # メッセージにユーザーからの質問を追加
    messages.append({"role": "user", "content": question.strip()})
    # やりとりが8を超えたら古いメッセージから削除
    if len(messages) > 8:
        del_message = messages.pop(0)

    # 言語モデルに質問
    response_message = process_response(question, tools)

    # メッセージに言語モデルからの回答を追加
    print(response_message, flush=True)
    messages.append({"role": "assistant", "content": response_message})

print("\n---ご利用ありがとうございました！---")


------define_tools(ツール定義)------


'質問:東京の見どころを教えて'

東京には多くの観光スポットがあります。以下にいくつかのおすすめの場所を挙げます。

1. **浅草寺** - 東京の最古の寺院で、雷門や仲見世通りの賑わいを楽しめます。
2. **東京スカイツリー** - 日本一高い塔からの壮大な景色を堪能できます。
3. **皇居** - 天皇の居所で、美しい庭園や外苑を散策できます。
4. **渋谷スクランブル交差点** - 世界で最も賑やかな交差点として知られています。
5. **秋葉原** - 電気製品やアニメ文化の中心地で、ショップ巡りが楽しめます。
6. **お台場** - 未来的な建物やレジャー施設が集中するエリアです。
7. **新宿御苑** - 都市の喧騒を離れ、静かな自然を楽しむことができる美しい公園です。
8. **銀座** - 高級ショッピングエリアで、グルメやアートも楽しめます。

さらに、以下のリンクも参考にしてみてください:
- [東京のおすすめスポット](https://travel.jr-central.co.jp/plan/area/tokyo/nav/11/)
- [日帰り旅行におすすめな東京近場の観光スポット](https://www.knt.co.jp/travelguide/kokunai/185/)

これらのスポットを巡ることで、東京の魅力を存分に体験できるでしょう！

---ご利用ありがとうございました！---
